In [1]:
import uproot
import awkward as ak #The events object is an awkward array
## Plotting.
import matplotlib.pyplot as plt

# Hists
import hist
from hist import Hist

# NanoEvents
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
# Processors
#import coffea.processor as processor

#Math
import math

#Numpy
import numpy as np

#File to read
fname='WPLEPWMHADjj_4f_EWK_UL2018-NANOAODSIMv9.root'

#Define the events
old_events = NanoEventsFactory.from_root(
    fname,
    schemaclass=NanoAODSchema,
    metadata={"dataset": "WPLEPWMHADjj_4f_EWK"}
).events()

In [2]:
#Check
print(ak.num(old_events.Electron))
print(old_events.Muon)
print(ak.size(old_events.Electron, axis=0))

[0, 0, 2, 1, 1, 1, 1, 0, 0, 0, 2, 0, 1, 0, ... 2, 1, 1, 1, 0, 0, 3, 0, 2, 1, 0, 1, 0]
[[], [Muon], [Muon], [], [], [Muon], [Muon, ... [], [Muon], [], [], [Muon], [], []]
500


In [23]:
#Implement the selections to veto events
from coffea.analysis_tools import PackedSelection

selection = PackedSelection()

#At least one lepton (boosted)
selection.add("one_l", ak.num(old_events.Electron)+ak.num(old_events.Muon)>=1 )

#Electron signal region
selection.add("one_tight_e", ak.mask(old_events.Electron, ak.num(old_events.Electron) > 1)[:, 0].mvaFall17V2Iso_WP90 == (3>2))
selection.add("no_2nd_tight_e", ak.mask(old_events.Electron, ak.num(old_events.Electron) > 1)[:, 1].mvaFall17V2Iso_WP90 == (3==2))
selection.add("loose_e", (ak.mask(old_events.Electron, ak.num(old_events.Electron) > 1)[:, 1].mvaFall17V2Iso_WPL == (3>2)) & (ak.mask(old_events.Electron, ak.num(old_events.Electron) > 1)[:, 1].pt >10))
selection.add("loose_mu1", (ak.mask(old_events.Muon, ak.num(old_events.Muon) > 1)[:, 0].looseId == (3>2)) & (ak.mask(old_events.Muon, ak.num(old_events.Muon) > 1)[:, 0].pt >10))
selection.add("loose_mu2", (ak.mask(old_events.Muon, ak.num(old_events.Muon) > 1)[:, 1].looseId == (3>2)) & (ak.mask(old_events.Muon, ak.num(old_events.Muon) > 1)[:, 1].pt >10))
selection.add("no_tight_mu", ak.any(old_events.Muon.tightId == (3==2), axis=1))
selection.add(
    "electron_pt",
    ak.any(old_events.Electron.pt >= 35.0, axis=1)  
)
selection.add(
    "e_range_eta",
    ak.any(abs (old_events.Electron.eta) < 2.5, axis=1)
)

#Muon signal region (boosted)
selection.add("one_tight_mu", ak.any(old_events.Muon.tightId == (3>2), axis=1))
selection.add("no_2nd_tight_mu", ak.mask(old_events.Muon, ak.num(old_events.Muon) > 1)[:, 1].tightId == (3==2))
selection.add("loose_mu", (ak.mask(old_events.Muon, ak.num(old_events.Muon) > 1)[:, 1].looseId == (3>2)) & (ak.mask(old_events.Muon, ak.num(old_events.Muon) > 1)[:, 1].pt >10))
selection.add("loose_e1", (ak.mask(old_events.Electron, ak.num(old_events.Electron) > 1)[:, 0].mvaFall17V2Iso_WPL == (3>2)) & (ak.mask(old_events.Electron, ak.num(old_events.Electron) > 1)[:, 0].pt >10))
selection.add("loose_e2", (ak.mask(old_events.Electron, ak.num(old_events.Electron) > 1)[:, 1].mvaFall17V2Iso_WPL == (3>2)) & (ak.mask(old_events.Electron, ak.num(old_events.Electron) > 1)[:, 1].pt >10))
selection.add("no_tight_e", ak.any(old_events.Electron.mvaFall17V2Iso_WP90 == (3==2), axis=1))
selection.add(
    "muon_Pt",
    ak.any(old_events.Muon.pt >= 30.0, axis=1)  
)
selection.add(
    "mu_range_eta",
    ak.any(abs (old_events.Muon.eta) < 2.4, axis=1)
)

#Puppy met cut
selection.add("lead_pMET", old_events.PuppiMET.pt>30)

#VBS jets (looking for the max invariant mass pair)
# Get all combinations of jet pairs in every event
dijets = ak.combinations(old_events.Jet, 2, fields=['i0', 'i1'])
# Check that jet pairs have the greatest mass
ismax=(dijets['i0']+dijets['i1']).mass==ak.max((dijets['i0']+dijets['i1']).mass, axis=1)
#previous line: returns a Boolean array with True where the condition is met, and false otherwise
# Mask the dijets with the ismax to get dijets with the gratest mass
VBS_jets = dijets[ismax]
# Separate pairs into arrays of the leading and the trailing VBS jets in each pair.
VBS_jet_l, VBS_jet_t = ak.unzip(VBS_jets)

#VBS selection
selection.add(
    "lead_VBS_ljet",   
    ak.all(VBS_jet_l.pt>50, axis=1)
)
selection.add(
    "lead_VBS_tjet",   
    ak.all(VBS_jet_t.pt>30, axis=1)
)
selection.add(
    "lead_VBS_mass",   
    ak.all((VBS_jet_l+VBS_jet_t).mass>500, axis=1)
)
selection.add(
    "lead_VBS_eta",   
    ak.all(abs(VBS_jet_l.eta-VBS_jet_t.eta)>2.5, axis=1)
)
selection.add("one_fatjet", (ak.num(old_events.FatJet) == 1))
selection.add("two_jet", (ak.num(old_events.Jet) >= 2))

#Transverse mass of the W boson decaying leptonically
#With electrons
selection.add(
    "W_Tmass_e",
    ak.all(2*old_events.Electron.pt*old_events.MET.pt*(1-np.cos(old_events.Electron.phi-old_events.MET.phi))<185**2, axis=1)
)
#With muons
selection.add(
    "W_Tmass_mu",
    ak.all(2*old_events.Muon.pt*old_events.MET.pt*(1-np.cos(old_events.Muon.phi-old_events.MET.phi))<185**2, axis=1)
)

#Invariant mass of the W boson decaying hadronically
selection.add(
    "W_mass_fatjet",
    ak.all((old_events.FatJet.msoftdrop>70) & (old_events.FatJet.msoftdrop<115), axis=1)
)

print(selection.names)

#Electron mask
ev_mask_e = selection.require(one_l=True, one_tight_e=True, no_2nd_tight_e=True, loose_e=False, loose_mu1=False, loose_mu2=False,  no_tight_mu=True, electron_pt=True, e_range_eta=True, lead_pMET=True, lead_VBS_ljet=True, lead_VBS_tjet=True, lead_VBS_mass=True, lead_VBS_eta=True, one_fatjet=True, two_jet=True, W_Tmass_e=True, W_mass_fatjet=True)
print(ev_mask_e.sum())

#Muon mask
ev_mask_mu = selection.require(one_l=True, one_tight_mu=True, no_2nd_tight_mu=True, loose_mu=False, loose_e1=False, loose_e2=False, no_tight_e=True,electron_pt=True, e_range_eta=True, lead_pMET=True, lead_VBS_ljet=True, lead_VBS_tjet=True, lead_VBS_mass=True, lead_VBS_eta=True, one_fatjet=True, two_jet=True, W_Tmass_mu=True, W_mass_fatjet=True)
print(ev_mask_mu.sum())

#Apply the cuts on the events with the mask (chose eeither electron or muon signal region)
events=old_events[ev_mask_e]

['one_l', 'one_tight_e', 'no_2nd_tight_e', 'loose_e', 'loose_mu1', 'loose_mu2', 'no_tight_mu', 'electron_pt', 'e_range_eta', 'one_tight_mu', 'no_2nd_tight_mu', 'loose_mu', 'loose_e1', 'loose_e2', 'no_tight_e', 'muon_Pt', 'mu_range_eta', 'lead_pMET', 'lead_VBS_ljet', 'lead_VBS_tjet', 'lead_VBS_mass', 'lead_VBS_eta', 'one_fatjet', 'two_jet', 'W_Tmass_e', 'W_Tmass_mu', 'W_mass_fatjet']
0
0


In [12]:
#Just a check
print(ak.size(events.Electron, axis=0))
print(events.Electron.pt)
print(events.Muon.pt)
print(events.Electron.mvaFall17V2Iso_WP90)
print(events.Muon.tightId)
print(events.Electron.mvaFall17V2Iso_WPL)
print(events.Muon.looseId)
#plt.hist(ak.flatten(events.Electron.pt), bins=100, range=[0,100], label="$p_{T}^{e}$ [GeV]")

9
[[46.3, 20.2], [63, 8.4], [41.4, 9.36], ... [85, 8.31], [57.9, 16.9], [60.3, 12.6]]
[[26.8, 4.26], [4.81], [3.3], [58.2, ... [17.7, 3.31], [30.7, 16.3], [3.91, 3.24]]
[[True, False], [True, False], [True, ... False], [True, False], [True, False]]
[[False, False], [False], [False], ... False], [False, False], [False, False]]
[[True, False], [True, False], [True, ... False], [True, False], [True, False]]
[[False, True], [False], [True], ... False, True], [False, False], [False, True]]


In [13]:
#Define the fiducial region

#Electron, muon , jets, fatjets
electron = events.Electron
muon = events.Muon
jets=events.Jet
fatjets = events.FatJet

#Jets isolation from electrons
# Get all combinations of jets and electrons in every event
jets_e = ak.cartesian({"x": jets, "y": electron})
# Check that jets satisfy the isolation
jets_iso_e = ((jets_e["x"].eta-jets_e["y"].eta)**2+(jets_e["x"].phi-jets_e["y"].phi)**2>0.4**2)
# Mask the jets_e with the jets_iso to get jets isolated from electrons
jets_e = jets_e[jets_iso_e]
# Separate pairs into jets and electons, redefining the jets (but not the electrons)
jets, el = ak.unzip(jets_e)

#Jets isolation from muons
# Get all combinations of jets and muons in every event
jets_mu = ak.cartesian({"x": jets, "y": muon})
# Check that jets satisfy the isolation
jets_iso_mu = ((jets_mu["x"].eta-jets_mu["y"].eta)**2+(jets_mu["x"].phi-jets_mu["y"].phi)**2>0.4**2)
# Mask the jets_mu with the jets_iso_mu to get jets isolated from muons
jets_mu = jets_mu[jets_iso_mu]
# Separate pairs into jets and muons, redefining the jets (but not the muons)
jets, mu = ak.unzip(jets_mu)

#FatJets isolation from electrons
# Get all combinations of fatjets and electrons in every event
fatjets_e = ak.cartesian({"x": fatjets, "y": electron})
# Check that jets satisfy the isolation
fatjets_iso_e = ((fatjets_e["x"].eta-fatjets_e["y"].eta)**2+(fatjets_e["x"].phi-fatjets_e["y"].phi)**2>0.8**2)
# Mask the fatjets_e with the fatjets_iso to get fatjets isolated from electrons
fatjets_e = fatjets_e[fatjets_iso_e]
# Separate pairs into fatjets and electons, redefining the fatjets (but not the electrons)
fatjets, el = ak.unzip(fatjets_e)

#FatJets isolation from electrons
# Get all combinations of fatjets and muons in every event
fatjets_mu = ak.cartesian({"x": fatjets, "y": muon})
# Check that fatjets satisfy the isolation
fatjets_iso_mu = ((fatjets_mu["x"].eta-fatjets_mu["y"].eta)**2+(fatjets_mu["x"].phi-fatjets_mu["y"].phi)**2>0.8**2)
# Mask the fatjets_mu with the fatjets_iso_mu to get fatjets isolated from muons
fatjets_mu = fatjets_mu[fatjets_iso_mu]
# Separate pairs into jets and muons, redefining the jets (but not the muons)
fatjets, mu = ak.unzip(fatjets_mu)

#Jets cuts
jets_pt_cut = (abs (jets.eta) < 4.7)
jets_eta_cut = (jets.pt > 30)
jets = jets[jets_pt_cut&jets_eta_cut]

#FatJets cuts
fatjets_pt_cut = (abs (fatjets.eta) < 2.4)
fatjets_eta_cut = (fatjets.pt > 200)
fatjets = fatjets[fatjets_pt_cut&fatjets_eta_cut]

#Removing AK4(Jet) jets overlapping with AK8(FatJets) jets
# Get all combinations of jets and fatjets in every event
jets_fatjets = ak.cartesian({"x": jets, "y": fatjets})
# Check that jets satisfy the isolation
jets_iso_f = ((jets_fatjets["x"].eta-jets_fatjets["y"].eta)**2+(jets_fatjets["x"].phi-jets_fatjets["y"].phi)**2>0.8**2)
# Mask the jets_fatjets with the jets_iso_f to get jets isolated from fatjets
jets_fatjets = jets_fatjets[jets_iso_f]
# Separate pairs into jets and electons, redefining the jets (but not the fatjets)
jets, fj = ak.unzip(jets_fatjets)